In [106]:
from ase import ase, io, neighborlist
from ase.calculators.amber import Amber
from functools import partial
import numpy as np

In [107]:
from ase import Atoms
d = 1.1
co = Atoms('CO', positions=[(0, 0, 0), (0, 0, d)], pbc=True,calculator=Amber)

In [ ]:
# need a way to calculate charges...

In [52]:
co.get_atomic_numbers()

array([6, 8])

In [3]:
def md(timestep, sim_length, data_file, temp, box):
    
    current_state = init(data_file, timestep)
    t = 0
    while t < sim_length:
        energy, forces = get_ef(current_state, timestep, box)
        new_positions = get_new_positions(current_state, timestep)
        new_velocities = get_new_velocities(current_state, timestep)
        t = t + timestep
        out = [current_state, sample(current_state), energies, forces]
    
    print("MD simulation finished")

In [57]:
co.arrays

{'numbers': array([6, 8]), 'positions': array([[0. , 0. , 0. ],
        [0. , 0. , 1.1]])}

### TODO: 
- write get_positions_from_file function
- write get_new_positions function
- write get_new_velocities function

In [94]:
md_system = {'atoms': co, 'forces': np.zeros(co.get_velocities().shape)}

In [95]:
md_system['atoms'].get_positions()

array([[0. , 1. , 0. ],
       [0. , 0. , 1.1]])

In [110]:
co.get_charges()

PropertyNotImplementedError: 

In [111]:
cutoff = neighborlist.natural_cutoffs(co)
nl = neighborlist.NeighborList(cutoff, self_interaction=False, bothways=True)
nl.update(co)
matrix = nl.get_connectivity_matrix()

atoms = {}
for i, atom in enumerate(co):
    atoms[atom] = {
        'position': co.get_positions()[i],
        'velocity': co.get_velocities()[i],
        'mass': co.get_masses()[i],
        #'charge': co.get_charges()[i],
        'force': [0, 0, 0],
        'bonds': []
    }
    
    

In [103]:
for i in enumerate(co):
    print(i)

(0, Atom('C', [0.0, 1.0, 0.0], index=0))
(1, Atom('O', [0.0, 0.0, 1.1], index=1))


In [7]:
#initialize the system with random kinetic energy
def init(initial_md_system, temp):
    
    positions = initial_md_system['atoms'].get_positions()
    n_at = len(positions)
    
    #initialize velocities
    #maxwell-boltzmann standard distribution (from TMP Chem)
    velocities = initial_md_system['atoms'].get_velocities()
    sigma_base = math.sqrt(2.0 * 1.98E-3 * self.temperature / 3)
    for vel in velocities:
       sigma = sigma_base * 
    
    velocities = np.random.normal(0.0, sigma, 3)
    
    sumv = velocities.sum() / n_at
    sumv2 = sumv**2 / n_at
    fs = np.sqrt(3*temp/sumv2)
    velocities = (velocities - sumv)*fs
    prev_positions = positions - (velocities*timestep)
    
    state['positions'] = positions
    state['velocities'] = velocities
    state['prev_positions'] = prev_positions
    state['n'] = n_at
    
    return state

In [43]:
def get_radius_pbc(atom_1, atom_2, L):
    r = np.subtract(atom_1, atom_2)
    r_nearest = [a - L*round(a/L) for a in r]
    return np.sqrt(np.square(r_nearest).sum())

In [9]:
def get_ef(state, temp, box):
    energy = 0
    rc = box/2
    ecut =  4*(rc**-12 - rc**-6)
    n_at = state['n']
    forces = np.zeros(n_at)
    for i in range(n_at-1):
        for j in range(i+1,n_at):
            r = get_radius_pbc(state['positions'][i],state['positions'][j],box)
            if r < rc:
                r2i = 1 / (r**2)
                r6i = r2i ** 3 
                ff = (48 * r2i) * (r6i * (r6i - 0.5))
                f[i] += ff*r
                f[j] += -ff*r
                energy += 4*r6i*(r6i-1)-ecut
    
    return (energy, forces)

In [44]:
def verlet()

SyntaxError: invalid syntax (<ipython-input-44-760e9a708c54>, line 1)

In [30]:
xi = (3, 1, 3)
xj = (5, 5, 9)

r = np.subtract(xi,xj)

In [17]:
z

-2

In [20]:
L = 10

In [21]:
r = np.subtract(xi,xj)

In [31]:
rl = [a - L*round(a/L) for a in r]

In [36]:
rl

[-2, -4, 4]

In [29]:
r

array([-2, -4, -2])

In [39]:
np.square(rl).sum()

36

In [42]:
get_radius_pbc(xi,xj,10)

6.0

In [49]:
np.zeros((3,1))

array([[0.],
       [0.],
       [0.]])

In [73]:
co.get_masses()

array([12.011, 15.999])

In [82]:
x = co.get_velocities()

In [83]:
x.shape

(2, 3)

In [84]:
np.zeros(x.shape)

array([[0., 0., 0.],
       [0., 0., 0.]])

In [91]:
co.get_positions()

array([[0. , 0. , 0. ],
       [0. , 0. , 1.1]])

In [93]:
co.set_positions([[0,1,0],[0,0,1.1]])

In [102]:
co.get_masses()

array([12.011, 15.999])